# Frangi Vesselness Filter (WIP: mostly complete, but depends on the imagej-notebook snapshot. Mostly just a testing ground for the new features.)

In [ ]:
//load ImageJ
%classpath config resolver imagej.public https://maven.imagej.net/content/groups/public
%classpath add jar /home/gabe/code/imagej/imagej-common/target/imagej-common-0.27.0-SNAPSHOT.jar
%classpath add jar /home/gabe/code/gselzer/imagej-notebook/target/imagej-notebook-0.3.0-SNAPSHOT.jar
%classpath add mvn net.imagej imagej 2.0.0-rc-67

//create ImageJ object
ij = new net.imagej.ImageJ()

This `Op` implements the algorithm for detection of vessel- or tube-like structures in 2D and 3D images described Frangi et al 1998, and is called in `ImageJ-Ops` as follows:

In [ ]:
ij.op().help('frangiVesselness')

As an example we will use an [MRA of a human brain](https://commons.wikimedia.org/w/index.php?curid=5930234) (By SBarnes - Own work, CC BY-SA 3.0) 

In [ ]:
input = ij.scifio().datasetIO().open("https://upload.wikimedia.org/wikipedia/commons/6/66/Mra-mip.jpg")
input

Before we call the `Op` we have to define a few more parameters:

* The spacing parameter refers to a `double[]` that describes the physical distance between data points in the image. If this data is available then it should be used, otherwise [1, ... , 1] is usually acceptible

* The scale parameter refers to the pixel distance at which the filter operates. Larger structures will be found using larger scale parameters.

In [ ]:
import net.imglib2.type.numeric.real.DoubleType
import net.imagej.notebook.NotebookService.ValueScaling;

//spacing refers to the physical distance between pixels. The default setting is {1, 1, 1...}
spacing = [1, 1, 1] as double[]

//scales refers the the pixel distance at which the filter operates. Larger scales measure larger structures.
scale = 4

input_slice = ij.op().run("hyperSliceView", input, 2, 0)
actualOutput = ij.op().create().img(input_slice, new DoubleType())
ij.op().filter().frangiVesselness(actualOutput, input_slice, spacing, scale)

actualOutput

Some of these vessels are a little hard to see. Let's use a [`DefaultDatasetView`](http://javadoc.scijava.org/ImageJ/net/imagej/display/DefaultDatasetView.html) so that we can rescale the minimum and maximum of the image. Let's also change the LookUp Table (LUT) to give a bit more contrast to the vessels:

In [ ]:
import net.imagej.display.ColorTables
import net.imagej.axis.Axes

clampMin = 0.00001 as double
clampMax = 0.02 as double
    
view = ij.notebook().view(actualOutput, clampMin, clampMax)
view.setColorTable(ColorTables.ICE, 0)
view.rebuild()
view